In [ ]:
# NECESSARY IMPORTS

import sys
sys.path.insert(0, 'modules/')
import mercury as mr
import ee
import geemap.foliumap as geemap
import os
import datetime
import webbrowser
import create_ee_creds as cec
import txtreading as txtr
import landsat_init as l_init
import sentinel_init as s_init
import modis_init as m_init
import othersds_init as o_init
from colorama import Fore

In [ ]:
#INTERFACE INIT

mr.Markdown(text = '**🌎 GAIA - Geospatial & Aerial Images Analyser [BETA] 🌎**')
mr.Markdown(text = '**🌎 This program solution is not final and some details are potentially to be changed 🌎**')
mr.Markdown(text = '**🌏 Current version - 0.32b 🌏**')

text = 'OSINT-TECHNOLOGIES'
text_params = {
    'fontsize': 10,
    'fontcolor': 'black',
    'bold': True,
    'padding': '3px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '1px',
    'position': 'bottomleft',
}

sources_list = ["Google EE"]
control_panel_choice = mr.Select(value="Google EE", choices=sources_list, label="Choose the source control panel")

app  = mr.App(title="GAIA v0.32b [BETA] 🌐",
              description="Open-Source Geospatial & Aerial Images Analyser",
              show_code=False,
              show_prompt=False,
              continuous_update=True,
              static_notebook=False,
              show_sidebar=True,
              full_screen=True,
              allow_download=False)

if control_panel_choice.value == "Google EE":
    ee_user_guide = mr.Button(label="How to start Google EE")
    ee_reg = mr.Button(label="EE registration")
    ee_service_acc = mr.Button(label="Create EE service account")
    save_map_html = mr.Button(label="Save current map [HTML]")
    ds_info = mr.Checkbox(value=False, label='Show datasets technical info')
    date_one = mr.Text(value="2022-01-01", label="Enter start date (format YYYY-MM-DD)", rows=1)
    date_two = mr.Text(value="2022-01-02", label="Enter final date (format YYYY-MM-DD)", rows=1)
    max_cloud_covering = mr.Slider(value=80, label='Max cloud cover (%, may affect datasets quality)', min=0, max=100)
    datasets_landsat = mr.Checkbox(value=False, label="Show Landsat datasets")
    datasets_sentinel = mr.Checkbox(value=False, label="Show Sentinel datasets")
    datasets_modis = mr.Checkbox(value=False, label="Show MODIS datasets")
    datasets_other = mr.Checkbox(value=False, label='Show other datasets')


if ee_reg.clicked:
    webbrowser.open_new_tab('https://code.earthengine.google.com/register')

if ee_service_acc.clicked:
    webbrowser.open_new_tab('https://console.cloud.google.com/iam-admin/serviceaccounts/')

if str((cec.service_acc_filename).rstrip()) not in os.listdir():
    print(Fore.RED + '[File/directory missing error]. Your EE credentials .JSON file was not found in GAIA directory')
    mr.Stop()
else:
    key_file = (cec.service_acc_filename).rstrip()
    service_account = (cec.service_acc_mail).rstrip()
    credentials = ee.ServiceAccountCredentials(service_account, key_file)
    ee.Initialize(credentials)

In [ ]:
#INITIALIZING MAP
m = geemap.Map()
m.add_text(text, **text_params)

#ADDING BASEMAPS
m.add_basemap('SATELLITE')
m.add_basemap('TERRAIN')
m.add_basemap('HYBRID')

#INITIALIZING AND PLOTTING DATASETS
try:
    if datasets_landsat.value:
        l_init.landsat_initialization(m, date_one, date_two, max_cloud_covering)

    if datasets_sentinel.value:
        s_init.sentinel_initialization(m, date_one, date_two, max_cloud_covering)

    if datasets_modis.value:
        m_init.modis_initialization(m, date_one, date_two)

    if datasets_other.value:
        o_init.others_initialization(m, date_one, date_two)

except ee.EEException as error:
    print(Fore.RED + '[Map plotting/initialization error] - {}'.format(str(error)[17:]))
    pass


mr.Markdown(text = "#GOOGLE EARTH ENGINE MAP 🌐")
mr.Markdown(text = "**DATASETS TIME PERIOD: from {} to {}**".format(date_one.value, date_two.value))

if ds_info.value:
    txtr.ds_info_read()

if ee_user_guide.clicked:
    txtr.ee_guide_read()

In [ ]:
# CALLING MAP OBJECT

m

In [ ]:
# POST-PROCESSING FUNCTIONS

download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
today = datetime.datetime.today()
time = today.strftime("%Y%m%d-%H.%M.%S")
timestr = str(time)

if save_map_html.clicked:
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    html_file = os.path.join(download_dir, 'GAIA-{}.html'.format(timestr))
    m.to_html(filename=html_file, title='My Map', width='100%', height='880px')